In [0]:
%run "/Workspace/Users/sundaramloganathan@outlook.com/includes/storage_acc_config"

In [0]:
%run "../includes/folder_path"

In [0]:
dbutils.widgets.text("p_owner_name","")
v_owner_name = dbutils.widgets.get("p_owner_name")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

pit_stops_schema = StructType(
    [
        StructField("raceId", IntegerType(), True),
        StructField("driverId", IntegerType(), True),
        StructField("stop", IntegerType(), True)    ,
        StructField("lap", IntegerType(), True),
        StructField("time", StringType(), True),
        StructField("duration", StringType(), True),
        StructField("milliseconds", IntegerType(), True)
    ]
)

In [0]:
pit_stops_df = spark.read.schema(pit_stops_schema).option("multiLine", True).json(f"{bronze_folder_path}/pit_stops.json")

In [0]:
from pyspark.sql.functions import current_timestamp,lit

pit_stops_upd_df = pit_stops_df.withColumnRenamed("raceId", "race_id") \
                               .withColumnRenamed("driverId", "driver_id") \
                               .withColumn("ingested_by",lit(v_owner_name)) \
                               .withColumn("ingestion_date", current_timestamp())   

In [0]:
pit_stops_upd_df.write.mode("overwrite").option("header", True).parquet(f"{silver_folder_path}/pit_stops")

In [0]:
dbutils.notebook.exit("Success")